In [2]:
import cv2 
import os 
import glob
import numpy as np

import matplotlib.pyplot as plt 

from tqdm import tqdm 
from module.utils import imread
from module.disp_measure import convert_by_img

In [3]:
img_list = glob.glob(os.path.join(r'D:\UOS-SSaS Dropbox\02. Projects\2019 NIA 인공지능기반 시설물 스마트 모니터링 체계 구축\2020.08.14 변위탐지모듈 파이썬변환', 'P34_9', '*.jpg'))

In [ ]:
result = [] 
src_img = imread(img_list[0])

for num, img_path in enumerate(img_list): 
    dest_img = imread(img_path)
    result.append(convert_by_img(dest_img,
                                 src_img,
                                 min_rad = 35,
                                 max_rad = 45,
                                 param1 = 100,
                                 param2 = 16))
    print('image number', num)

94.27797
94.27797
x_dist 1.0 y_dist 190.0
113.12656
113.12656
x_dist 0.0 y_dist 0.0
image number 0
114.12248
114.12248
x_dist 1.0 y_dist 1.0
image number 1
113.32659
113.32659
x_dist 1.0 y_dist 0.0
image number 2
113.740616
113.740616
x_dist 1.0 y_dist 1.0
image number 3
112.13067
112.13067
x_dist 1.0 y_dist 0.0
image number 4
94.04797
94.04797
x_dist 4.0 y_dist 188.0
111.520584
111.520584
x_dist 0.0 y_dist 1.0
image number 5
114.713905
114.713905
x_dist 2.0 y_dist 0.0
image number 6
111.91497
111.91497
x_dist 2.0 y_dist 1.0
image number 7
113.93666
113.93666
x_dist 2.0 y_dist 1.0
image number 8
113.674866
113.674866
x_dist 1.0 y_dist 2.0
image number 9
113.690636
113.690636
x_dist 1.0 y_dist 2.0
image number 10
113.32659
113.32659
x_dist 1.0 y_dist 0.0
image number 11
112.720665
112.720665
x_dist 2.0 y_dist 3.0
image number 12
113.68138
113.68138
x_dist 1.0 y_dist 2.0
image number 13
112.23219
112.23219
x_dist 2.0 y_dist 1.0
image number 14
113.50455
113.50455
x_dist 0.0 y_dist 0.0
im

In [ ]:
collect_rslt = []
for rslt in result : 
    collect_rslt.append(rslt[0])

In [ ]:
plt.plot(np.array(collect_rslt))

In [9]:
np.savetxt("P34_7.csv", np.array(collect_rslt), delimiter=",")

In [ ]:
## check the circle detection accuracy 

param1 = 100
param2 = 11
min_rad = 35
max_rad = 50
img = imread(img_list[17]) 
grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret3, th3 = cv2.threshold(grey_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

circles = cv2.HoughCircles(th3, 
                               cv2.HOUGH_GRADIENT,
                               1, 
                               20, 
                               param1=param1,
                               param2=param2,
                               minRadius=min_rad,
                               maxRadius=max_rad)

# Draw circles that are detected. 
if circles is not None: 
  
    # Convert the circle parameters a, b and r to integers. 
    circles = np.uint16(np.around(circles)) 
  
    for pt in circles[0, :]: 
        a, b, r = pt[0], pt[1], pt[2] 
  
        # Draw the circumference of the circle. 
        cv2.circle(th3, (a, b), r, (0, 255, 0), 2) 
  
        # Draw a small circle (of radius 1) to show the center. 
        cv2.circle(th3, (a, b), 1, (0, 0, 255), 3) 
        cv2.imshow("Detected Circle", th3) 
        cv2.waitKey(0) 
else:
        cv2.imshow('Circle is not detected', th3)
        cv2.waitKey(0) 